<a href="https://colab.research.google.com/github/nedokormysh/GB_ML_business/blob/course_project/GB_ML_business_no_docker_request.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Строим запрос к нашему сервису

In [144]:
import requests

import urllib.request
import json 

In [148]:
# Пример данных
data = ('city_136', 0.827, 'Male', 'No relevent experience', 'Full time course', 'Masters', 'STEM', '4', '50-99', 'Funded Startup', '1', 11)

In [149]:
# формируем запрос
def send_json(x):
    city, city_development_index, gender, relevent_experience, enrolled_university, education_level, major_discipline, experience, company_size, company_type, last_new_job, training_hours = x
    
    body = {
            'city': city, 
            'city_development_index': city_development_index,
            'gender': gender,
            'relevent_experience': relevent_experience,
            'enrolled_university': enrolled_university,
            'education_level': education_level,
            'major_discipline': major_discipline,
            'experience': experience,
            'company_size': company_size,
            'company_type': company_type,
            'last_new_job': last_new_job,
            'training_hours': training_hours,
        }

    # print(body)

    myurl = 'http://8cd2-35-245-59-110.ngrok.io' + '/predict'
    headers = {'content-type': 'application/json; charset=utf-8'}
    response = requests.post(myurl, json=body, headers=headers)
    return response.json()['predictions']

In [150]:
# обращение к серверу с запросом из одного набора (его построили руками выше - data)
response = send_json(data)
print('предсказание', response)

предсказание 0.07990615


Сделаем обработку массы запросов:

1. загрузим данные X_test

In [151]:
import pandas as pd
from sklearn.metrics import roc_auc_score, roc_curve, precision_recall_curve
from sklearn.metrics import f1_score
from urllib import request, parse
import numpy as np


X_test = pd.read_csv("https://raw.githubusercontent.com/nedokormysh/GB_ML_business/course_project/X_test.csv")
y_test = pd.read_csv("https://raw.githubusercontent.com/nedokormysh/GB_ML_business/course_project/y_test.csv")

In [152]:
X_test['training_hours'] = X_test['training_hours'].astype(float)

2. Передаем запрос одиночный из наших данных и посмотрим на ответ

In [154]:
X_test[['city',
                             'city_development_index',
                             'gender',
                             'relevent_experience',
                             'enrolled_university',
                             'education_level',
                             'major_discipline',
                             'experience',
                             'company_size',
                             'company_type',
                             'last_new_job',
                             'training_hours']].iloc[0,:]

city                                    city_103
city_development_index                      0.92
gender                                       NaN
relevent_experience       No relevent experience
enrolled_university                no_enrollment
education_level                      High School
major_discipline                             NaN
experience                                    <1
company_size                                 NaN
company_type                                 NaN
last_new_job                                 NaN
training_hours                               6.0
Name: 0, dtype: object

In [155]:
response = send_json(X_test[['city',
                             'city_development_index',
                             'gender',
                             'relevent_experience',
                             'enrolled_university',
                             'education_level',
                             'major_discipline',
                             'experience',
                             'company_size',
                             'company_type',
                             'last_new_job',
                             'training_hours']].iloc[0,:])

In [156]:
response

'0.33687592'

3. Сделаем N запросов и оценим время

In [157]:
N = 100

In [158]:
%%time
predictions = X_test[
                     ['city',
                             'city_development_index',
                             'gender',
                             'relevent_experience',
                             'enrolled_university',
                             'education_level',
                             'major_discipline',
                             'experience',
                             'company_size',
                             'company_type',
                             'last_new_job',
                             'training_hours']
                     ].iloc[:N].apply(lambda x: send_json(x), axis=1)

CPU times: user 413 ms, sys: 56.4 ms, total: 469 ms
Wall time: 28.8 s


In [159]:
predictions = predictions.astype(float)

4. Посчитаем метрику

In [160]:
y_test.head()

,target
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [161]:
roc_auc_score(y_score=predictions.values, y_true=y_test.values[:N])

0.78125